# Coffea and Awkward

**March 15, 2023 at Beijing**

shift + command + k = delete line
command + / = annotating code

In [1]:
import awkward as ak
import uproot
import hist
import os
import numpy as np
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.nanoevents.methods.base import NanoEventsArray
from imp import reload #每次修改调用函数后可以更新
import correctionlib, rich
import matplotlib.pyplot as plt
import sys
from coffea.lumi_tools import LumiMask

/tmp/ipykernel_199071/3220351898.py:8: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload #每次修改调用函数后可以更新


### coffea读文件

In [2]:
import input_file 
reload(input_file)
# test_events = input_file.input_file('test')# 
###############  DATASETS
##### Electron channel
# dataC_ele_events = input_file.input_file('dataC_ele')# 15min
# dataD_ele_events = input_file.input_file('dataD_ele')# 5min
# dataE_ele_events = input_file.input_file('dataE_ele')# 8min
# dataF_ele_events = input_file.input_file('dataF_ele')# 26min
# dataG_ele_events = input_file.input_file('dataG_ele')# 4min

##### Muon channel
# dataC_muon_events = input_file.input_file('dataC_muon')# 11min
# dataD_muon_events = input_file.input_file('dataD_muon')# 5min
dataE_muon_events = input_file.input_file('dataE_muon')# 5min 
# dataF_muon_events = input_file.input_file('dataF_muon')# --------
dataG_muon_events = input_file.input_file('dataG_muon')# 4min


###############  MC
##### 2022
# zg0_22_events = input_file.input_file('zg0_22')# 5min
# zg1_22_events = input_file.input_file('zg1_22')# 1min part!!!
# zg2_22_events = input_file.input_file('zg2_22')# 1min
# zg3_22_events = input_file.input_file('zg3_22')# 1min
# ttbar_22_events = input_file.input_file('tt_22')# 2min
# wz_22_events = input_file.input_file('wz_22')# 2min
# ww_22_events = input_file.input_file('ww_22')# 1min
# zz_22_events = input_file.input_file('zz_22')# 1min
# dy_22_events = input_file.input_file('dy_22')# 2min

##### 2022EE
# zg0_22EE_events = input_file.input_file('zg0_22EE')# 5min
# zg1_22EE_events = input_file.input_file('zg1_22EE')# 2min part!!!
# zg2_22EE_events = input_file.input_file('zg2_22EE')# 1min
# zg3_22EE_events = input_file.input_file('zg3_22EE')# 1min
# ttbar_22EE_events = input_file.input_file('tt_22EE')# 4min
# wz_22EE_events = input_file.input_file('wz_22EE')# 2min30
# ww_22EE_events = input_file.input_file('ww_22EE')# 1min
# zz_22EE_events = input_file.input_file('zz_22EE')# 1min
# dy_22EE_events = input_file.input_file('dy_22EE')# ----------

#muon channel 19min
#ele channel 

dataE_muon  is done!
dataG_muon  is done!


In [3]:
EVENT_ELE = {
    # 'test': test_events,
    # 'dataC_ele':dataC_ele_events,
    # 'dataD_ele':dataD_ele_events,
    'dataE_ele':dataE_ele_events,
    # # 'dataF':dataF_events,
    'dataG_ele':dataG_ele_events,

    # 'zg0_22': zg0_22_events,
    # 'zg1_22': zg1_22_events,
    # 'zg2_22': zg2_22_events,
    # 'zg3_22': zg3_22_events,
    # 'tt_22': ttbar_22_events,
    # 'zz_22': zz_22_events, 
    # 'wz_22': wz_22_events,
    # 'ww_22': ww_22_events,  
    # 'dy_22': dy_22_events,

    'zg0_22EE': zg0_22EE_events,
    'zg1_22EE': zg1_22EE_events,
    'zg2_22EE': zg2_22EE_events,
    'zg3_22EE': zg3_22EE_events,
    'tt_22EE': ttbar_22EE_events,
    'zz_22EE': zz_22EE_events, #No LHEpdf
    'wz_22EE': wz_22EE_events,
    'ww_22EE': ww_22EE_events,  
    'dy_22EE': dy_22EE_events,

}

In [3]:
EVENT_MUON = {
    # 'test': test_events,
    # 'dataC_muon':dataC_muon_events,
    # 'dataD_muon':dataD_muon_events,
    'dataE_muon':dataE_muon_events,
    # 'dataF':dataF_events,
    'dataG_muon':dataG_muon_events,

    # 'zg0_22': zg0_22_events,
    # 'zg1_22': zg1_22_events,
    # 'zg2_22': zg2_22_events,
    # 'zg3_22': zg3_22_events,
    # 'tt_22': ttbar_22_events,
    # 'zz_22': zz_22_events,
    # 'wz_22': wz_22_events,
    # 'ww_22': ww_22_events, 
    # 'dy_22': dy_22_events, #一个要求prompt，一个要求nonprompt

    # 'zg0_22EE': zg0_22EE_events,
    # 'zg1_22EE': zg1_22EE_events,
    # 'zg2_22EE': zg2_22EE_events,
    # 'zg3_22EE': zg3_22EE_events,
    # 'tt_22EE': ttbar_22EE_events,
    # 'zz_22EE': zz_22EE_events, #No LHEpdf
    # 'wz_22EE': wz_22EE_events,
    # 'ww_22EE': ww_22EE_events,  
    # 'dy_22EE': dy_22EE_events,

}

## selection

In [ ]:
import selection_cut
reload(selection_cut)
event_ele_phodrs = {key: {} for key in EVENT_ELE}

for key in EVENT_ELE.keys():
    event_ele_variables = []
    for i in EVENT_ELE[key]:
        if 'data' not in key:
            event_weight = {'event_weight':np.sign(i.Generator.weight)}
        elif 'dataC' in key:
            i = selection_cut.run_num(i,'C')
            event_weight =  {'event_weight':'NONE'}
        elif 'dataD' in key:
            i = selection_cut.run_num(i,'D')
            event_weight =  {'event_weight':'NONE'}
        elif 'dataE' in key:
            i = selection_cut.run_num(i,'E')
            event_weight =  {'event_weight':'NONE'}
        # elif key == 'dataF':
        #     murun = selection_cut.muon(i,'run_num','F')
        #     event_weight =  {'event_weight':'NONE'}
        #     mu0 = selection_cut.muon(murun,'num',True)
        elif key == 'dataG':
            i = selection_cut.run_num(i,'G')
            event_weight =  {'event_weight':'NONE'}
        leptons,photons,event = selection_cut.electron_nTuple(i,key,"A")
        # mass_ee = selection_cut.MASS_Ele(leptons,photons,'ee')
        # mass_gee = selection_cut.MASS_Ele(leptons,photons,'gee')
        # event_ele_variables.append(selection_cut.VARIABLES('ele',key,leptons,photons,event,mass_ee,mass_gee,event_weight))    
        event_ele_variables.append(selection_cut.VARIABLES('ele',key,leptons,photons,event,False,False,event_weight))    


    event_ele_phodrs[key] = event_ele_variables
    print(key,'event_phodrs',event_ele_phodrs[key][0])




#dataC_ele 54min
#dataD_ele 4min
#zg0_22 #19min
#zg1_22 2min
#zg2_22 1min
#zg3_22 1min
#tt_22 25min
#ww,wz,zz 5min
#total 113min

zg0_22 event_phodrs {'photon_pt': <Array [37.3, 41.1, 30.6, ... 33.6, 36.8, 32.9] type='293 * ?float32[parameters=...'>, 'LHEPdfWeight': <Array [[1, 1, 0.996, ... 1, 0.958, 1.04]] type='293 * option[[var * float32[par...'>, 'LHEScaleWeight': <Array [[1.08, 1.19, 1.28, ... 0.885, 0.955]] type='293 * option[[var * float32[...'>, 'PSWeight': <Array [[1.01, 0.98, 0.969, ... 1.28, 2.21]] type='293 * option[[var * float32[p...'>}
zg1_22 event_phodrs {'photon_pt': <Array [81.9, 55.3, 49.8, ... 69.8, 81.6, 74.5] type='15627 * ?float32[parameter...'>, 'LHEPdfWeight': <Array [[1, 0.996, 1, 1, ... 1, 0.971, 1.03]] type='15627 * option[[var * float3...'>, 'LHEScaleWeight': <Array [[0.92, 0.975, 1.06, ... 0.854, 0.993]] type='15627 * option[[var * float...'>, 'PSWeight': <Array [[0.716, 0.944, 1.48, ... 1.22, 0.614]] type='15627 * option[[var * float...'>}
zg2_22 event_phodrs {'photon_pt': <Array [261, 127, 105, 149, ... 97.5, 106, 125] type='549 * ?float32[parameters=...'>, 'LHEPdfWeight': <Array 

In [5]:
# ele!!!!
# sample_names = ["dataC_ele","dataD_ele","zg0_22","zg1_22", "zg2_22", "zg3_22","tt_22","zz_22", "ww_22", "wz_22","dy_22"]#
# sample_names = ["zg0_22","zg1_22", "zg2_22", "zg3_22"]#,
sample_names = ["zg0_22","zg1_22", "zg2_22", "zg3_22","tt_22"]#"tt",
# sample_names = ["dataC_ele","dataD_ele"]#,"dy""zg2", "zg3"

# 初始化一个字典，用于存储合并后的数据
event_ele_final = {}

# 遍历每个样本名称
for sample_name in sample_names:
    print(sample_name)
    # 初始化一个字典，用于存储当前样本的合并数据
    event_ele_final[sample_name] = {}

    # 获取第一个条目中的信息键
    info_keys = list(event_ele_phodrs[sample_name][0].keys())

    # 遍历每个信息键
    for info_key in info_keys:
        # 从 event_muon_phodrs 中提取当前样本和信息键的数据
        data_to_concatenate = [entry[info_key] for entry in event_ele_phodrs[sample_name]]

        # 使用 ak.concatenate 将数据连接在一起
        event_ele_final[sample_name][info_key] = ak.concatenate(data_to_concatenate)


#all 95min

zg0_22


zg1_22
zg2_22
zg3_22
tt_22


In [13]:
event_ele_final

NameError: name 'event_ele_final' is not defined

In [9]:
def save_event_dict(event_dict, file_prefix):
    # data_subkeys = ['photon_pt', 'photon_eta', 'ele1_pt', 'ele2_pt', 'ele1_eta', 'ele2_eta', 'ele_mass','gee_mass','photon_endcap','photon_barrel']#,'npvsGood','Rho_Calo','Rho_tracker']#,,,'dr_lg','charge1','charge2'
    data_subkeys = ['photon_pt']#['photon_r9','photon_seedGain','run']#'npvsGood','Rho_Calo','Rho_tracker'#'photon_endcap','photon_barrel'
    other_subkeys = data_subkeys + ['LHEPdfWeight','LHEScaleWeight','PSWeight']#['generator_weight', 'event_weight']#'LHEPdfWeight','LHEScaleWeight','PSWeight','GenDressedLepton','GenIsolatedPhoton','gen_gmumu','gen_mumu' 'gen_ele1_pt','gen_ele2_pt','gen_ele1_eta','gen_ele2_eta','gen_gmumu','gen_mumu',,'gen_gmumu'
    other_subkeys = data_subkeys 

    for key, value in event_dict.items():
        subkeys = other_subkeys if 'data' not in key else data_subkeys
        for subkey in subkeys:
            ak.to_parquet(value[subkey], f'{file_prefix}_{key}_{subkey}.parquet')



In [10]:
# 存储数据
save_event_dict(event_ele_final, 'selection_ele_file/v2/A/event_ele_final')
# save_event_dict(event_ele_final, 'selection_ele_file/v2/B/event_ele_final')
# save_event_dict(event_ele_final, 'selection_ele_file/v2/C/event_ele_final')
# save_event_dict(event_ele_final, 'selection_ele_file/v2/D/event_ele_final')

# save_event_dict(event_ele_final, 'ABCD/ABCD_file/ele_channel/A/event_ele_final')

In [4]:
import selection_cut
reload(selection_cut)
event_muon_phodrs = {key: {} for key in EVENT_MUON}

for key in EVENT_MUON.keys():
    event_muon_variables = []
    for i in EVENT_MUON[key]:
        if 'data' not in key:
            event_weight = {'event_weight':np.sign(i.Generator.weight)}
        elif 'dataC' in key:
            i = selection_cut.run_num(i,'C')
            event_weight =  {'event_weight':'NONE'}
        elif 'dataD' in key:
            i = selection_cut.run_num(i,'D')
            event_weight =  {'event_weight':'NONE'}
        elif 'dataE' in key:
            i = selection_cut.run_num(i,'E')
            event_weight =  {'event_weight':'NONE'}
        elif 'dataF' in key:
            i = selection_cut.run_num(i,'F')
            event_weight =  {'event_weight':'NONE'}
        elif key == 'dataG':
            i = selection_cut.run_num(i,'G')
            event_weight =  {'event_weight':'NONE'}
        # leptons,photons,event = selection_cut.muon_nTuple(i,key,"A")
        # mass_mu = selection_cut.MASS_Muon(leptons,photons,'mu')
        # mass_gmumu = selection_cut.MASS_Muon(leptons,photons,'gmumu')
        # event_muon_variables.append(selection_cut.VARIABLES('muon',key,leptons,photons,event,mass_mu,mass_gmumu,event_weight))   
#ABCD
        leptons,photons,event = selection_cut.muon_nTuple(i,key,"A")
        event_muon_variables.append(selection_cut.VARIABLES('muon',key,leptons,photons,event,False,False,event_weight))   


    event_muon_phodrs[key] = event_muon_variables
    print(key,'event_phodrs',event_muon_phodrs[key][0])



#dataC 28min
#dataD 12min 
#zg0_22 #20min
#zg1_22 2min
#zg2_22 1min
#zg3_22 1min
#tt_22 25min
#ww,wz,zz 5min
#total_CD 94min
#total_EG 

dataE_muon event_phodrs {'photon_pt': <Array [56.6, 57.4, 64.2, 33.4] type='4 * ?float32[parameters={"__doc__": "pt"}]'>, 'photon_eta': <Array [-1.02, 0.445, 0.56, -1.18] type='4 * ?float32[parameters={"__doc__": "et...'>, 'muon1_pt': <Array [58.5, 112, 67.6, 35.7] type='4 * ?float32[parameters={"__doc__": "pt"}]'>, 'muon2_pt': <Array [34.7, 33.3, 53.1, 31.8] type='4 * ?float32[parameters={"__doc__": "pt"}]'>, 'muon1_eta': <Array [-0.388, 0.124, -0.785, 0.891] type='4 * ?float32[parameters={"__doc__": ...'>, 'muon2_eta': <Array [0.353, -0.575, -1.54, -0.787] type='4 * ?float32[parameters={"__doc__": ...'>}
dataG_muon event_phodrs {'photon_pt': <Array [44.3, 52.3, 34.4, ... 51, 53.5, 81.4] type='9 * ?float32[parameters={"__...'>, 'photon_eta': <Array [1.03, -0.719, -1.6, ... 0.758, 1.62] type='9 * ?float32[parameters={"__d...'>, 'muon1_pt': <Array [45.4, 73.5, 65, ... 67.1, 50, 40.1] type='9 * ?float32[parameters={"__do...'>, 'muon2_pt': <Array [42.9, 20, 26.1, ... 52.2, 49.3, 28.9] typ

In [5]:
# muon!!!!
# sample_names = ["dataC_muon","dataD_muon","zg0_22","zg1_22", "zg2_22", "zg3_22","tt_22","zz_22", "ww_22", "wz_22","dy_22"]
# sample_names = ["dataE_muon","dataG_muon","zg0_22EE","zg1_22EE", "zg2_22EE", "zg3_22EE","tt_22EE","zz_22EE", "ww_22EE", "wz_22EE","dy_22EE"]
# sample_names = ["zg0_22","zg1_22", "zg2_22", "zg3_22","tt_22"]#,,"zz_22", "ww_22", "wz_22","dy_22"
sample_names = ["dataE_muon","dataG_muon"]#"tt",
# sample_names = ["dataC_muon","dataD_muon"]

# 初始化一个字典，用于存储合并后的数据
event_muon_final = {}

# 遍历每个样本名称
for sample_name in sample_names:
    print(sample_name)
    # 初始化一个字典，用于存储当前样本的合并数据
    event_muon_final[sample_name] = {}

    # 获取第一个条目中的信息键
    info_keys = list(event_muon_phodrs[sample_name][0].keys())

    # 遍历每个信息键
    for info_key in info_keys:
        # 从 event_muon_phodrs 中提取当前样本和信息键的数据
        data_to_concatenate = [entry[info_key] for entry in event_muon_phodrs[sample_name]]

        # 使用 ak.concatenate 将数据连接在一起
        event_muon_final[sample_name][info_key] = ak.concatenate(data_to_concatenate)

#dataE 1s
#zg1,zg2,zg3 15min
#tt 70min
#zz,ww,wz 20min

dataE_muon
dataG_muon


In [6]:
event_muon_final

{'dataE_muon': {'photon_pt': <Array [56.6, 57.4, 64.2, ... 30.3, 31.2, 34.7] type='1665 * ?float32[parameters...'>,
  'photon_eta': <Array [-1.02, 0.445, 0.56, ... 0.549, -0.212] type='1665 * ?float32[parameters=...'>,
  'muon1_pt': <Array [58.5, 112, 67.6, ... 36.6, 44, 43.4] type='1665 * ?float32[parameters={"...'>,
  'muon2_pt': <Array [34.7, 33.3, 53.1, ... 29.9, 38.8, 32] type='1665 * ?float32[parameters={...'>,
  'muon1_eta': <Array [-0.388, 0.124, ... -0.764, -0.69] type='1665 * ?float32[parameters={"__d...'>,
  'muon2_eta': <Array [0.353, -0.575, -1.54, ... 0.129, -2.07] type='1665 * ?float32[parameters...'>},
 'dataG_muon': {'photon_pt': <Array [44.3, 52.3, 34.4, ... 88.9, 35.2, 49] type='697 * ?float32[parameters={"...'>,
  'photon_eta': <Array [1.03, -0.719, -1.6, ... 1.8, -0.364] type='697 * ?float32[parameters={"_...'>,
  'muon1_pt': <Array [45.4, 73.5, 65, ... 82.5, 33.9, 65.5] type='697 * ?float32[parameters={"...'>,
  'muon2_pt': <Array [42.9, 20, 26.1, ... 34.4, 24.2, 

In [7]:
def save_event_dict(event_dict, file_prefix):
    # data_subkeys = ['photon_pt', 'photon_eta', 'muon1_pt', 'muon2_pt', 'muon1_eta', 'muon2_eta', 'mu_mass','gmumu_mass']#'photon_endcap','photon_barrel','npvsGood','Rho_Calo','Rho_tracker',,'dr_lg','charge1','charge2'
    data_subkeys =  ['photon_pt', 'photon_eta', 'muon1_pt', 'muon2_pt', 'muon1_eta', 'muon2_eta', ]#'mu_mass','gmumu_mass']#['photon_r9','photon_seedGain','run']#['npvsGood','Rho_Calo','Rho_tracker']
    # other_subkeys = data_subkeys + ['generator_weight', 'event_weight']#['nPU']#['LHEPdfWeight','LHEScaleWeight','PSWeight']#'LHEPdfWeight','LHEScaleWeight','PSWeight','GenDressedLepton','GenIsolatedPhoton','gen_gmumu','gen_mumu' 'gen_muon1_pt','gen_muon2_pt','gen_muon1_eta','gen_muon2_eta','gen_gmumu','gen_mumu','LHEPdfWeight','LHEScaleWeight',,'gen_gmumu'
    other_subkeys = data_subkeys

    for key, value in event_dict.items():
        subkeys = other_subkeys if 'data' not in key else data_subkeys
        for subkey in subkeys:
            ak.to_parquet(value[subkey], f'{file_prefix}_{key}_{subkey}.parquet')



In [8]:
# 存储数据
# save_event_dict(event_muon_final, '../selection_muon_file/v2/A/event_muon_final')
# save_event_dict(event_muon_final, 'selection_muon_file/v2/B/event_muon_final')
# save_event_dict(event_muon_final, 'selection_muon_file/v2/C/event_muon_final')
# save_event_dict(event_muon_final, 'selection_muon_file/v2/D/event_muon_final')


save_event_dict(event_muon_final, '../selection_EFG_muon_file/A/event_muon_final')

# Test

In [20]:
print(EVENT_ELE['zg0_22'][0].Photon.pixelSeed)

[[False], [], [], [True, False], ... False], [], [False], [True, True, False]]


In [22]:
print(EVENT_ELE['zg0_22'][0].Photon.pt)

[[12.8], [], [], [29.4, 21.3], [15.4], ... [12.4], [], [13.8], [37.7, 27.1, 10.7]]


In [224]:
###ELECTRON

import selection_cut
reload(selection_cut)
# sample = EVENT_ELE['dataD_ele'][10]
# Tag = 'dataD'
# sample = EVENT_ELE['wz_22'][0]
# Tag = 'wz'
sample = EVENT_ELE['zg1_22'][0]
Tag = 'zg1_22'
ABCD = "A"
# print('Start (Electron channel, sample:',Tag, ', region:',ABCD,') ',len(sample))
# ele0 = selection_cut.electron_nTuple(sample,Tag,ABCD)[0]
# print('HLT:',len(ele0),'Efficiency:',f"{len(ele0)/len(sample) * 100}%")
# ele1 = selection_cut.electron_nTuple(sample,Tag,ABCD)[1]
# ele1_ele = selection_cut.electron_nTuple(sample,Tag,ABCD)[2]
# print('pt, eta, medium id, charge, two ele:',len(ele1),'Efficiency:',f"{len(ele1)/len(ele0) * 100}%")
# ele2 = selection_cut.electron_nTuple(sample,Tag,ABCD)[3]
# ele2_ele = selection_cut.electron_nTuple(sample,Tag,ABCD)[4]
# print('Z mass window:',len(ele2),'Efficiency:',f"{len(ele2)/len(ele1) * 100}%")
# pho1 = selection_cut.electron_nTuple(sample,Tag,ABCD)[5]
# pho1_ele = selection_cut.electron_nTuple(sample,Tag,ABCD)[6]
# pho1_pho = selection_cut.electron_nTuple(sample,Tag,ABCD)[7]
# print('Photon pt,eta,id, at least one photon:',len(pho1),'Efficiency:',f"{len(pho1)/len(ele2) * 100}%")
# pho2 = selection_cut.electron_nTuple(sample,Tag,ABCD)[8]
# pho2_ele = selection_cut.electron_nTuple(sample,Tag,ABCD)[9]
# pho2_pho = selection_cut.electron_nTuple(sample,Tag,ABCD)[10]
# print('Photon lepton dR, at least one photon:',len(pho2),'Efficiency:',f"{len(pho2)/len(pho1) * 100}%")
# pho3 = selection_cut.electron_nTuple(sample,Tag,ABCD)[11]
# pho3_ele = selection_cut.electron_nTuple(sample,Tag,ABCD)[12]
# pho3_pho = selection_cut.electron_nTuple(sample,Tag,ABCD)[13]
# print('Photon prompt (pixelSeed), at least one photon:',len(pho3),'Efficiency:',f"{len(pho3)/len(pho2) * 100}%")
# pho4 = selection_cut.electron_nTuple(sample,Tag,ABCD)[14]
# pho4_ele = selection_cut.electron_nTuple(sample,Tag,ABCD)[15]
# pho4_pho = selection_cut.electron_nTuple(sample,Tag,ABCD)[16]
# print('Photon prompt (Flav), at least one photon:',len(pho4),'Efficiency:',f"{len(pho4)/len(pho3) * 100}%")
# pho5 = selection_cut.electron_nTuple(sample,Tag,ABCD)[17]
# pho5_ele = selection_cut.electron_nTuple(sample,Tag,ABCD)[18]
# pho5_pho = selection_cut.electron_nTuple(sample,Tag,ABCD)[19]
# print('Photon prompt (dR), only one photon:',len(pho5),'Efficiency:',f"{len(pho5)/len(pho4) * 100}%")

pho5_ele = selection_cut.electron_nTuple(sample,Tag,ABCD)[0]
pho5_pho = selection_cut.electron_nTuple(sample,Tag,ABCD)[1]
print('End',len(pho5_ele))


End 15627


In [227]:
### MUON
import selection_cut
reload(selection_cut)
# sample = EVENT_MUON['dataD_muon'][10]
# Tag = 'dataD'
# sample = EVENT_MUON['wz_22'][0]
# Tag = 'wz'
sample = EVENT_MUON['zg1_22'][0]
Tag = 'zg1_22'
ABCD = 'A'
# print('Start (Muon channel, sample:',Tag, ', region:',ABCD,') ',len(sample))
# mu0 = selection_cut.muon_nTuple(sample,Tag,ABCD)[0]
# print('HLT:',len(mu0),'Efficiency:',f"{len(mu0)/len(sample) * 100}%")
# mu1 = selection_cut.muon_nTuple(sample,Tag,ABCD)[1]
# mu1_mu = selection_cut.muon_nTuple(sample,Tag,ABCD)[2]
# print('pt, eta, tight id&iso, charge, two mu:',len(mu1),'Efficiency:',f"{len(mu1)/len(mu0) * 100}%")
# mu2 = selection_cut.muon_nTuple(sample,Tag,ABCD)[3]
# mu2_mu = selection_cut.muon_nTuple(sample,Tag,ABCD)[4]
# print('Z mass window:',len(mu2),'Efficiency:',f"{len(mu2)/len(mu1) * 100}%")
# pho1 = selection_cut.muon_nTuple(sample,Tag,ABCD)[5]
# pho1_mu = selection_cut.muon_nTuple(sample,Tag,ABCD)[6]
# pho1_pho = selection_cut.muon_nTuple(sample,Tag,ABCD)[7]
# print('Photon pt,eta,i, at least one photon:',len(pho1),'Efficiency:',f"{len(pho1)/len(mu2) * 100}%")
# pho2 = selection_cut.muon_nTuple(sample,Tag,ABCD)[8]
# pho2_mu = selection_cut.muon_nTuple(sample,Tag,ABCD)[9]
# pho2_pho = selection_cut.muon_nTuple(sample,Tag,ABCD)[10]
# print('Photon lepton dR, at least one photon:',len(pho2),'Efficiency:',f"{len(pho2)/len(pho1) * 100}%")
# pho3 = selection_cut.muon_nTuple(sample,Tag,ABCD)[11]
# pho3_mu = selection_cut.muon_nTuple(sample,Tag,ABCD)[12]
# pho3_pho = selection_cut.muon_nTuple(sample,Tag,ABCD)[13]
# print('Photon prompt (pixelSeed), at least one photon:',len(pho3),'Efficiency:',f"{len(pho3)/len(pho2) * 100}%")
# pho4 = selection_cut.muon_nTuple(sample,Tag,ABCD)[14]
# pho4_mu = selection_cut.muon_nTuple(sample,Tag,ABCD)[15]
# pho4_pho = selection_cut.muon_nTuple(sample,Tag,ABCD)[16]
# print('Photon prompt (Flav), at least one photon:',len(pho4),'Efficiency:',f"{len(pho4)/len(pho3) * 100}%")
# pho5 = selection_cut.muon_nTuple(sample,Tag,ABCD)[17]
# pho5_mu = selection_cut.muon_nTuple(sample,Tag,ABCD)[18]
# pho5_pho = selection_cut.muon_nTuple(sample,Tag,ABCD)[19]
# print('Photon prompt (dR), at least one photon:',len(pho5),'Efficiency:',f"{len(pho5)/len(pho4) * 100}%")

pho5_mu = selection_cut.muon_nTuple(sample,Tag,ABCD)[0]
pho5_pho = selection_cut.muon_nTuple(sample,Tag,ABCD)[1]
print('End',len(pho5_mu))


End 22032


In [205]:
ak.num(pho5_pho)

<Array [1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1] type='116 * ?int64'>

In [232]:
pho4.Muon.eta

<Array [[0.123, 0.872], ... [0.314, -0.471]] type='1836 * option[var * float32[p...'>

In [233]:
pho4_mu.eta

<Array [[0.123, 0.872], ... [0.314, -0.471]] type='1836 * option[var * float32[p...'>

In [234]:
pho4.Photon.eta

<Array [[0.857, -2.68], ... 0.46], [0.399]] type='1836 * option[var * float32[pa...'>

In [235]:
pho4_pho.eta

<Array [[0.857], [-2.03], ... [0.46], [0.399]] type='1836 * option[var * float32...'>

In [17]:
import selection_cut
reload(selection_cut)
event_phodrs = {key: {} for key in EVENT}


for key in EVENT.keys():
    event_variables = []
    # fiducial_variables = []
    for i in EVENT[key]:
#muon_cut
        if 'data' not in key:
            event_weight = {'event_weight':np.sign(i.Generator.weight)}
            mu0 = selection_cut.muon(i,'num',True)
        elif key == 'dataF':
            murun = selection_cut.muon(i,'run_num','F')
            event_weight =  {'event_weight':'NONE'}
            mu0 = selection_cut.muon(murun,'num',True)
        elif key == 'dataG':
            murun = selection_cut.muon(i,'run_num','G')
            event_weight =  {'event_weight':'NONE'}
            mu0 = selection_cut.muon(murun,'num',True)            
        mu1 = selection_cut.muon(mu0,'trigger','double')
        mu2,event2 = selection_cut.muon(mu1,'cutBasedID','tight')
        mu3 = selection_cut.muon(mu2,'pf','tight')
        mu4 = selection_cut.muon(mu3,'eta','2.4')
        mu5 = selection_cut.muon(mu4,'pt',event2)
        mu6 = selection_cut.muon(mu5,'pid',True)
        mass_mumu = selection_cut.MASS(mu6,'mu')
        mu_mass = selection_cut.muon(mu6,'mass',mass_mumu)
#photon_cut
        pho1 = selection_cut.photon(mu_mass, 'pt', '30')
        pho2 = selection_cut.photon(pho1, 'eta', '2.5')
        pho3 = selection_cut.photon(pho2, 'id', 'tight') #tight #B #C #D
        if 'data' in key:
            # event_variables.append(selection_cut.VARIABLES(pho3,mass_mumu,mass_gmumu,key,event_weight))
            lg_near, lg_dr = pho3.Photon.nearest(pho3.Muon, axis=1, return_metric=True)
            fsr = selection_cut.muon(pho3,'fsr',lg_dr)
            mass_mumu = selection_cut.MASS(fsr,'mu')
            mass_gmumu = selection_cut.MASS(fsr,'gmumu')
            event_variables.append(selection_cut.VARIABLES(fsr,mass_mumu,mass_gmumu,False, False,key,event_weight))
        else:
            lg_near, lg_dr = pho3.Photon.nearest(pho3.Muon, axis=1, return_metric=True)
            fsr = selection_cut.muon(pho3,'fsr',lg_dr)
            # pho4 = selection_cut.photon(pho3,'prompt',True) #看细节
            # pho4 = selection_cut.photon(pho3,'prompt',False) #看细节
            # gen_photon = pho4.GenPart[(pho4.GenPart.pdgId==22) & pho4.GenPart.hasFlags(['isLastCopy',"isPrompt"])]
            # photon4_near, photon4_gen_photon_dr = gen_photon.nearest(pho4.Photon, axis=1, return_metric=True)
            # mask = ak.fill_none(photon4_gen_photon_dr < 0.3, True)
            # pho5 = pho4[ak.num(pho4.Photon[ak.any(mask == True,axis=1)]) == 1]
            # lg_near, lg_dr = pho5.Photon.nearest(pho5.Muon, axis=1, return_metric=True)
            # fsr = selection_cut.muon(pho5,'fsr',lg_dr)
            mass_mumu = selection_cut.MASS(fsr,'mu')
            mass_gmumu = selection_cut.MASS(fsr,'gmumu')
            event_variables.append(selection_cut.VARIABLES(fsr,mass_mumu,mass_gmumu,key,event_weight))
            # gen_gmumu = selection_cut.gen_mass(fsr,'gmumu')
            # gen_mumu = selection_cut.gen_mass(fsr,'mu')
            # event_variables.append(selection_cut.VARIABLES(fsr,mass_mumu,mass_gmumu,gen_gmumu,gen_mumu,key,event_weight))
    event_phodrs[key] = event_variables
    print(key,'event_phodrs',event_phodrs[key][0])



#dataF 74m  
#dataG 10m
#zg0 30m
#mc 98m

zg0 event_phodrs {'photon_pt': <Array [[42], [36], ... [32.6], [33.8]] type='188 * option[var * float32[paramet...'>, 'photon_endcap': <Array [[False], [False], ... [True], [False]] type='188 * option[var * bool[par...'>, 'photon_barrel': <Array [[True], [True], ... [False], [True]] type='188 * option[var * bool[param...'>, 'Photon_genPartFlav': <Array [[1], [1], [1], ... [1], [1], [1]] type='188 * option[var * uint8[paramet...'>, 'Gen_statusflag': <Array [[10625, 22913, 4481, ... 12352, 12352]] type='188 * option[var * int32[p...'>, 'generator_weight': <Array [1, 1, 1, -1, 1, 1, ... -1, 1, 1, 1, 1] type='188 * ?float32'>, 'event_weight': <Array [1, 1, 1, -1, 1, 1, ... 1, -1, 1, 1, 1] type='202914 * float32'>}
zg1 event_phodrs {'photon_pt': <Array [[95.3], [53.3], ... [86.5], [56.1]] type='521 * option[var * float32[par...'>, 'photon_endcap': <Array [[True], [False], ... [True], [True]] type='521 * option[var * bool[param...'>, 'photon_barrel': <Array [[False], [True], ... [False], [Fa

In [18]:
#清理内存
# import gc

# del EVENT
# gc.collect()

In [19]:
# 样本名称列表
# sample_names = ["dataF", "dataG","zg1", "zg2", "zg3", "tt", "zz", "ww", "wz"]
# sample_names = ["zg0"]
sample_names = ["zg0","zg1", "zg2", "zg3"]
# sample_names = ["tt", "zz", "ww", "wz"]#"tt",
# sample_names = ["dataF", "dataG"]
# sample_names = ["dy1","dy2"]#,"dy""zg2", "zg3"

# 初始化一个字典，用于存储合并后的数据
event_final = {}

# 遍历每个样本名称
for sample_name in sample_names:
    print(sample_name)
    # 初始化一个字典，用于存储当前样本的合并数据
    event_final[sample_name] = {}

    # 获取第一个条目中的信息键
    info_keys = list(event_phodrs[sample_name][0].keys())

    # 遍历每个信息键
    for info_key in info_keys:
        # 从 event_muon_phodrs 中提取当前样本和信息键的数据
        data_to_concatenate = [entry[info_key] for entry in event_phodrs[sample_name]]

        # 使用 ak.concatenate 将数据连接在一起
        event_final[sample_name][info_key] = ak.concatenate(data_to_concatenate)

#zg0 10min
#zg1 
#dy1 dy2 277min

zg0
zg1
zg2
zg3


In [20]:
def save_event_dict(event_dict, file_prefix):
    # data_subkeys = ['photon_pt', 'photon_eta', 'muon1_pt', 'muon2_pt', 'muon1_eta', 'muon2_eta', 'muon_mass','gmumu_mass','photon_endcap','photon_barrel','npvsGood','Rho_Calo','Rho_tracker']#,'npvsGood','Rho_Calo','Rho_tracker',,'dr_lg','charge1','charge2'
    data_subkeys = ['Photon_genPartFlav','Gen_statusflag','photon_pt','photon_endcap','photon_barrel',]
    other_subkeys = data_subkeys + ['generator_weight', 'event_weight',]#'LHEPdfWeight','LHEScaleWeight','PSWeight','GenDressedLepton','GenIsolatedPhoton','gen_gmumu','gen_mumu' 'gen_muon1_pt','gen_muon2_pt','gen_muon1_eta','gen_muon2_eta','gen_gmumu','gen_mumu','LHEPdfWeight','LHEScaleWeight',,'gen_gmumu'
    # other_subkeys = data_subkeys

    for key, value in event_dict.items():
        subkeys = other_subkeys if 'data' not in key else data_subkeys
        for subkey in subkeys:
            ak.to_parquet(value[subkey], f'{file_prefix}_{key}_{subkey}.parquet')



In [21]:
# 存储数据
# save_event_dict(event_final, 'selection_file/event_final')
save_event_dict(event_final, 'ABCD_file/closure_test/A/event_final')
# save_event_dict(event_final, 'ABCD_file/B/event_final')
# save_event_dict(event_final, 'test_file/D/event_final')

# TEST

In [12]:
sample = EVENT['zg3'][0]
pass_mu = (sample['GenDressedLepton'].pt > 10) 
pass_pho = (sample['GenIsolatedPhoton'].pt > 10)
sample_tem_test = sample['GenDressedLepton'][abs(sample['GenDressedLepton'].eta) < 2.4]#2.4
in_fiducial_1 = sample[(ak.num(pass_pho,axis=1)==1) & (ak.num(pass_mu,axis=1)==2)]
out_fiducial_1 = sample[~((ak.num(pass_pho,axis=1)==1) & (ak.num(pass_mu,axis=1)==2))]
in_fiducial_2 = in_fiducial_1[ak.num(sample_tem.eta)==2]
out_fiducial_2 = in_fiducial_1[ak.num(sample_tem.eta)!=2]


out_fiducial_all = sample[(~((ak.num(pass_pho,axis=1)==1) & (ak.num(pass_mu,axis=1)==2))) | (ak.num(sample_tem_test.eta)!=2)]

In [13]:
print(len(in_fiducial_1))
print(len(out_fiducial_1))
print(len(out_fiducial_all))

292
347
356


In [10]:
sample = EVENT['zg3'][0]
pass_mu = (sample['GenDressedLepton'].pt > 10) 
pass_pho = (sample['GenIsolatedPhoton'].pt > 10)
in_fiducial_1 = sample[(ak.num(pass_pho,axis=1)==1) & (ak.num(pass_mu,axis=1)==2)]
out_fiducial_1 = sample[~((ak.num(pass_pho,axis=1)==1) & (ak.num(pass_mu,axis=1)==2))]
sample_tem = in_fiducial_1['GenDressedLepton'][abs(in_fiducial_1['GenDressedLepton'].eta) < 2.4]#2.4
in_fiducial_2 = in_fiducial_1[ak.num(sample_tem.eta)==2]
out_fiducial_2 = in_fiducial_1[ak.num(sample_tem.eta)!=2]

In [6]:
print(len(in_fiducial_1))
print(len(out_fiducial_1))
print(len(in_fiducial_2))
print(len(out_fiducial_2))

292
347
283
9


In [6]:
import selection_cut
reload(selection_cut)

i = EVENT['zg3'][0]
event_weight = {'event_weight':np.sign(i.Generator.weight)}
mu0 = selection_cut.muon(i,'num',True)          
mu1 = selection_cut.muon(mu0,'trigger','double')
mu2,event2 = selection_cut.muon(mu1,'cutBasedID','tight')
mu3 = selection_cut.muon(mu2,'pf','tight')
mu4 = selection_cut.muon(mu3,'eta','2.4')
mu5 = selection_cut.muon(mu4,'pt',event2)
mu6 = selection_cut.muon(mu5,'pid',True)
mass_mumu = selection_cut.MASS(mu6,'mu')
mu_mass = selection_cut.muon(mu6,'mass',mass_mumu)
#photon_cut
pho1 = selection_cut.photon(mu_mass, 'pt', '30')
pho2 = selection_cut.photon(pho1, 'eta', '2.5')
pho3 = selection_cut.photon(pho2, 'id', 'tight') #tight
pho4 = selection_cut.photon(pho3,'prompt',True)
gen_photon = pho4.GenPart[(pho4.GenPart.pdgId==22) & pho4.GenPart.hasFlags(['isLastCopy',"isPrompt"])]
photon4_near, photon4_gen_photon_dr = gen_photon.nearest(pho4.Photon, axis=1, return_metric=True)
mask = ak.fill_none(photon4_gen_photon_dr < 0.3, True)
pho5 = pho4[ak.num(pho4.Photon[ak.any(mask == True,axis=1)]) == 1]
# event_variables.append(selection_cut.VARIABLES(pho5,mass_mumu,mass_gmumu,key,event_weight))
lg_near, lg_dr = pho5.Photon.nearest(pho5.Muon, axis=1, return_metric=True)
fsr = selection_cut.muon(pho5,'fsr',lg_dr)
gen_mumu = selection_cut.gen_mass(fsr,'mu')
gen_mumu


#dataF 74m  
#dataG 10m
#zg0 30m
#mc 98m

<Array [83.8, 91.6, 89.6, ... 90.3, 90.1, 92.4] type='30 * ?float32'>

In [58]:
gen_mumu = selection_cut.gen_mass(fsr,'mu')

In [57]:
ak.max(samples['GenDressedLepton']['pdgId'])

13

In [53]:
gen_muon_vectors_0 = ak.zip(
    {
        "pt": samples['GenDressedLepton']['pt'],
        "eta": samples['GenDressedLepton']['eta'],
        "phi": samples['GenDressedLepton']['phi'],
        "mass": samples['GenDressedLepton']['mass']
    },
    with_name="PtEtaPhiMLorentzVector"
)
# GEN_gamma = GEN[GEN['pdgId'] == 22]
gen_photon_vector_0 = ak.zip(
    {
        "pt": samples['GenIsolatedPhoton']['pt'],
        "eta": samples['GenIsolatedPhoton']['eta'],
        "phi": samples['GenIsolatedPhoton']['phi'],
        "mass": samples['GenIsolatedPhoton']['mass'],
    },
    with_name="PtEtaPhiMLorentzVector"
)
gen_muon_vectors = gen_muon_vectors_0[ak.num(gen_muon_vectors_0)==2]
gen_photon_vector = gen_photon_vector_0[ak.num(gen_muon_vectors_0)==2]
# gen_muon_vectors = gen_muon_vectors_0[ak.num(gen_muon_vectors_0)==2][ak.num(gen_photon_vector_0['pt'])==1]
# gen_photon_vector = gen_photon_vector_0[ak.num(gen_muon_vectors_0)==2][ak.num(gen_photon_vector_0['pt'])==1]

In [65]:
samples = fsr
gen_muon_vectors_0 = ak.zip(
        {
            "pt": samples['GenDressedLepton']['pt'],
            "eta": samples['GenDressedLepton']['eta'],
            "phi": samples['GenDressedLepton']['phi'],
            "mass": samples['GenDressedLepton']['mass']
        },
        with_name="PtEtaPhiMLorentzVector"
    )
gen_muon_vectors = gen_muon_vectors_0[ak.num(gen_muon_vectors_0)==2]
# GEN_gamma = GEN[GEN['pdgId'] == 22]
gen_photon_vector = ak.zip(
    {
        "pt": fsr['GenIsolatedPhoton']['pt'],
        "eta": fsr['GenIsolatedPhoton']['eta'],
        "phi": fsr['GenIsolatedPhoton']['phi'],
        "mass": fsr['GenIsolatedPhoton']['mass'],
    },
    with_name="PtEtaPhiMLorentzVector"
)
invariant_mass = (gen_muon_vectors[:, 0] + gen_muon_vectors[:, 1] ).mass
# invariant_mass = (gen_muon_vectors[:, 0] + gen_muon_vectors[:, 1] + gen_photon_vector[:,0]).mass
invariant_mass

<Array [85.2, 92.7, 91.4, ... 83.2, 89.6, 85.7] type='89 * ?float32'>

In [63]:
samples = fsr
gen_muon_idxs = samples['Muon']["genPartIdx"]
GEN = samples['GenPart']
gen_muon_vectors = ak.zip(
    {
        "pt": GEN[gen_muon_idxs]['pt'],
        "eta": GEN[gen_muon_idxs]['eta'],
        "phi": GEN[gen_muon_idxs]['phi'],
        "mass": GEN[gen_muon_idxs]['mass']
    },
    with_name="PtEtaPhiMLorentzVector"
)
# 获取gen level photon的四动量信息
gen_photon_idx = samples.Photon[:, 0]["genPartIdx"]
GEN = ak.flatten(samples['GenPart'])
gen_photon_vector = ak.zip(
    {
        "pt": GEN[gen_photon_idx]['pt'],
        "eta": GEN[gen_photon_idx]['eta'],
        "phi": GEN[gen_photon_idx]['phi'],
        "mass": GEN[gen_photon_idx]['mass'],
    },
    with_name="PtEtaPhiMLorentzVector"
)
invariant_mass = (gen_muon_vectors[:, 0] + gen_muon_vectors[:, 1] ).mass
invariant_mass

<Array [92.6, 90.9, 90.1, ... 93.5, 92.6, 90.4] type='6499 * ?float32'>

In [66]:
        GEN = samples['GenPart']
        GEN_muons = GEN[abs(GEN['pdgId']) == 13]
        GEN_gamma = GEN[GEN['pdgId'] == 22]
        GEN_VARIABLES = {
            'gen_muon1_pt':GEN_muons['pt'][:,0],
            'gen_muon2_pt':GEN_muons['pt'][:,1],
            'gen_muon1_eta':GEN_muons['eta'][:,0],
            'gen_muon2_eta':GEN_muons['eta'][:,1],
            'gen_gamma_pt':GEN_gamma['pt'][:,0],
            'gen_gamma_eta':GEN_gamma['eta'][:,0],
            }

/home/pku/muty/anaconda3/lib/python3.8/site-packages/awkward/_connect/_numpy.py:195: RuntimeWarning: overflow encountered in sinh
  result = getattr(ufunc, method)(
/home/pku/muty/anaconda3/lib/python3.8/site-packages/awkward/_connect/_numpy.py:195: RuntimeWarning: invalid value encountered in multiply
  result = getattr(ufunc, method)(
/home/pku/muty/anaconda3/lib/python3.8/site-packages/awkward/_connect/_numpy.py:195: RuntimeWarning: overflow encountered in cosh
  result = getattr(ufunc, method)(


<Array [671, nan, nan, 400, ... nan, nan, 815] type='6499 * ?float32'>